## Demo 3.2: Vcluster without Datastore Persistence

### Create vcluster

##### Values.yaml preparation

In [1]:
! cat values.yaml

# Refer https://github.com/loft-sh/vcluster/blob/main/charts/k3s/values.yaml for full set of customisable options

storage:
  persistence: false

##### Create vcluster with the customized values.yaml

In [2]:
! vcluster create vc1 -n host-namespace -f values.yaml --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/3803531792 --namespace host-namespace --install --repository-config='' --values /tmp/2449566535 --values values.yaml
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [8]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-wjbsr-x-kube-system-x-vc1   1/1     Running   0          5s
pod/vc1-0                                          2/2     Running   0          15s

NAME                                        TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1        ClusterIP      10.96.14.244   <none>           53/UDP,53/TCP,9153/TCP   5s
service/vc1                                 LoadBalancer   10.96.222.44   172.18.255.201   443:31452/TCP            15s
service/vc1-headless                        ClusterIP      None           <none>           443/TCP                  15s
service/vc1-node-kcd-vcluster-demo-worker   ClusterIP      10.96.103.25   <none>           10250/TCP                5s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     15s


##### Verify that persistence is disabled

######  By verifying the helm values

In [9]:
! echo "storage.persistence:" $(helm get values vc1 -n host-namespace -a | grep -v COMPUTED_VALUES | yq '.storage.persistence')

storage.persistence: false


######  By verifying vc1-0 pod's data volume

In [10]:
! kubectl get pod vc1-0 --namespace host-namespace \
  -o jsonpath="{.spec.volumes[?(@.name == 'data')]}" | jq

{
  "emptyDir": {},
  "name": "data"
}


##### Accessing Vcluster

In [11]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


##### Create a workload inside the vcluster

In [12]:
! kubectl create ns ns1 --kubeconfig ./kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


In [15]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-d4r2h   1/1     Running   0          6s
sample-7f75ccc884-5f9qj   1/1     Running   0          6s


##### Simulate a vcluster failure by deleting vcluster pod in host cluster

In [16]:
 ! kubectl get pods -n host-namespace

NAME                                           READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-wjbsr-x-kube-system-x-vc1   1/1     Running   0          66s
sample-7f75ccc884-5f9qj-x-ns1-x-vc1            1/1     Running   0          18s
sample-7f75ccc884-d4r2h-x-ns1-x-vc1            1/1     Running   0          18s
vc1-0                                          2/2     Running   0          76s


In [17]:
 ! kubectl delete pod vc1-0 -n host-namespace

pod "vc1-0" deleted


In [29]:
! kubectl get pods -n host-namespace

NAME                                           READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-6n98p-x-kube-system-x-vc1   1/1     Running   0          44s
vc1-0                                          2/2     Running   0          56s


In [30]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

Unable to connect to the server: x509: certificate signed by unknown authority


In [31]:
! vcluster connect vc1 --namespace host-namespace  --kube-config new_kubeconfig.yaml

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: new_kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig new_kubeconfig.yaml get namespaces`


In [32]:
! kubectl get pods -n ns1 --kubeconfig ./new_kubeconfig.yaml 

No resources found in ns1 namespace.


### Cleanup

In [33]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/385538987 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
namespace "host-namespace" deleted


In [34]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   36m
kube-node-lease      Active   36m
kube-public          Active   36m
kube-system          Active   36m
local-path-storage   Active   36m
metallb-system       Active   35m
